# **Model Fine-tuning using Annotated Datasets**

## **A. Preliminaries**

These are several libraries needed to install as the source code has dependencies on such.

In [ ]:
pip install pandas openpyxl

In [ ]:
# Mounting Drive for loading .xlsx files and saving models
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"       # Set to true to avoid API prompt during fine-tuning

## **B. Fine-tuning**

Note that for the following, it is assumed that Google Drive is mounted along with the datasets (train, dev,and test) mounted at the root directory. This also assumes a directory named "MODEL" exists in the root directory, where each models during the grid search is saved for documentation and experimentation purposes.

It is also advised to use the T4 GPU for running the said code, as it hastens the training and optimizations greatly. The expected duration of the sources code's runtime is apporximately 4 hours using the T4 GPU.

Furhter details concerning the section of code is discussed using the code's comments.

An overview of the whole code is outlined here:

1. The dataset (.xlsx) subdivided into *train, dev* and *test* is loaded in a dataframe then into a single dataset dictionaty utilizing the `load_excel_data()` function.
2. Relation labels are mapped to integers, in this case from (0,1,2)
3. The parameter values are prepared for hyperparameter optimization
4. A table or array for `results` per iteration during the grid search is initialized.
5. The Grid search starts with varying combinations of learning rate and batch sizes.
6. For training the models, the data is first formatted by masking the specfici entity into its entity type, as observed in `preprocessing data`. Essentially, there is no difference between the formatting for SpanBERT and RoBERTa since this modification was only recent due to following such strategy of conventional formatting for RE fine-tuning. This newly formatted chunk is saved in a column named: `input_text`
7. After formatting, all other columns aer dropped except `input_text` and `labels`. Note that the `labels` column comprise of the ID of corresponding to the a unique relation label.
8. This then proceeds to Tokenization, which both models share the same module as for BERT.
9. The pre-trained model is then initialized from HuggingFace, afterwards then proceeds into training with the specified parameters. The trained and fine-tuned model is also saved in persistent memory afterwards
10. After each training and validation, the fine-tuned model is then evaluated with the assigned metrics using the test set and logged into the `results` array, in which now the iteration loops to train another model with different parameters.
11. After all iterations, the results are printed out in one table for analysis and discussion.

In [ ]:
import itertools
import shutil
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset, DatasetDict
import pandas as pd
import torch

# Labeled Dataset loaded from Google drive
train_file = "/content/drive/MyDrive/final_train.xlsx"
dev_file = "/content/drive/MyDrive/final_dev.xlsx"
test_file = "/content/drive/MyDrive/final_test.xlsx"

# Models from HuggingFace transformers
SPANBERT_MODEL = "SpanBERT/spanbert-base-cased"
ROBERTA_MODEL = "roberta-base"

# Function for loading the excel files into
# dataframs and into one dictionary
def load_excel_data(train_file, dev_file, test_file):
    def process_file(file_path):

        df = pd.read_excel(file_path)

        # Extract desired columns (entity pairs and relation labels + three-sentence chunks)
        df = df.rename(columns={
            "Chunk": "chunk",
            "Entity 1": "entity1",
            "Entity 2": "entity2",
            "Entity 1 Type": "entity_type1",
            "Entity 2 Type": "entity_type2",
            "relation": "relation"
        })

        return Dataset.from_pandas(df)

    datasets = DatasetDict({
        "train": process_file(train_file),
        "dev": process_file(dev_file),
        "test": process_file(test_file)
    })
    return datasets


# Preprocessing the data
# Preparing it for input formatting for both models
def preprocess_data(batch, tokenizer, model_name, label_to_id, max_seq_length):
    inputs = []
    for chunk, entity1, entity2, entity_type1, entity_type2 in zip(
        batch["chunk"], batch["entity1"], batch["entity2"], batch["entity_type1"], batch["entity_type2"]
    ):
        if "spanbert" in model_name:
            input_text = chunk.replace(entity1, f"[{entity_type1}]").replace(entity2, f"[{entity_type2}]") # SpanBERT-specific input formatting: Similar for BERT formattings
        else:
            input_text = chunk.replace(entity1, f"[{entity_type1}]").replace(entity2, f"[{entity_type2}]") # RoBERTa-specific input formatting: not really different since this was updated to follow the procedure for input formattings for BERT models
        inputs.append(input_text)

    encodings = tokenizer(inputs, padding=True, truncation=True, max_length=max_seq_length)
    encodings["labels"] = [label_to_id[label] for label in batch["relation"]]
    return encodings

# Function for performance evaluation using scikit-metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# The General Training Function
def train_model(model_name, tokenizer, datasets, output_dir, best_model_save_path, num_labels, preprocess_fn, max_seq_length, lr, batch_size):
    # Tokenize datasets
    tokenized_datasets = datasets.map(
        lambda batch: preprocess_fn(batch, tokenizer, model_name, label_to_id, max_seq_length),   # This line use the preporcessing and formatting as defined earlier,
        batched=True
    )
    tokenized_datasets = tokenized_datasets.remove_columns(["chunk", "entity1", "entity2", "relation"])  # Keep only input and labels (2-columns) after formatting

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)   # Loading the model from HuggingFace using the path specified

    # Training arguments and parameters
    # which are repeatedly modified during optimization
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=8,                        # 8 epochs for all iterations
        weight_decay=0.01,
        save_total_limit=2,
        logging_dir=f"{output_dir}/logs",
        load_best_model_at_end=True,
        metric_for_best_model="f1",                # f1 here is set as the metric
        fp16=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["dev"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # Train and save
    trainer.train()
    trainer.save_model(output_dir)  # Save the last model to output_dir

    # Save the best model to the specific location
    trainer.save_model(best_model_save_path)  # Save the best model explicitly

    # Optional: Move the best model to a specific location using shutil
    specific_location = "/content/drive/MyDrive/MODEL/"  # model saving path in drive
    if os.path.exists(best_model_save_path):
        shutil.move(best_model_save_path, os.path.join(specific_location, os.path.basename(best_model_save_path)))

    # Evaluate on test set
    test_results = trainer.evaluate(tokenized_datasets["test"])
    return test_results



# MAIN CODE STARTS HERE
datasets = load_excel_data(train_file, dev_file, test_file)

# This line here maps the labels into integer IDs:
# For instance, 0: `has_taxon`, ....
label_to_id = {label: idx for idx, label in enumerate(set(datasets["train"]["relation"]))}
for split in datasets.keys():
    datasets[split] = datasets[split].map(lambda example: {"label": label_to_id[example["relation"]]})

# Hyperparameters to be tested using grid search
max_seq_length = 128
learning_rates = [5e-6, 1e-5, 2e-5, 3e-5, 5e-5]
batch_sizes = [16, 32]
hyperparameter_combinations = list(itertools.product(learning_rates, batch_sizes))

# Train Models with Hyperparameter Tuning
results_summary = []

for model_name, tokenizer_name, model_output_dir in [
    (SPANBERT_MODEL, SPANBERT_MODEL, "./spanbert_model"),
    (ROBERTA_MODEL, ROBERTA_MODEL, "./roberta_model")
]:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    print(f"Now training {model_name}")

    for lr, batch_size in hyperparameter_combinations:
        print(f"Experimenting {model_name} with LR={lr}, Batch Size={batch_size}")

        # Train the model
        best_model_path = f"{model_output_dir}_best_lr{lr}_bs{batch_size}"
        test_results = train_model(
            model_name=model_name,
            tokenizer=tokenizer,
            datasets=datasets,
            output_dir=f"{model_output_dir}_lr{lr}_bs{batch_size}",
            best_model_save_path=best_model_path,
            num_labels=len(label_to_id),
            preprocess_fn=preprocess_data,
            max_seq_length=max_seq_length,
            lr=lr,
            batch_size=batch_size,
        )

        # Log results
        results_summary.append({
            "model": model_name,
            "learning_rate": lr,
            "batch_size": batch_size,
            "accuracy": test_results["eval_accuracy"],
            "f1": test_results["eval_f1"],
            "precision": test_results["eval_precision"],
            "recall": test_results["eval_recall"],
        })

# Print the Summary stored in the results array
import pandas as pd
results_df = pd.DataFrame(results_summary)
print(results_df)

/usr/local/lib/python3.10/dist-packages/datasets/table.py:719: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  return cls(pa.Table.from_pandas(*args, **kwargs))


Map:   0%|          | 0/3128 [00:00<?, ? examples/s]

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-6-731bacd33aee>:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.657289,0.669118,0.678827,0.665506,0.714724
2,No log,0.815458,0.610294,0.608139,0.739361,0.598285
3,No log,1.029751,0.661765,0.663086,0.666663,0.689344
4,No log,1.021951,0.632353,0.610099,0.663078,0.623314
5,No log,0.691882,0.713235,0.694339,0.691917,0.699363
6,0.380500,1.221044,0.625000,0.615216,0.664620,0.647083
7,0.380500,1.306088,0.639706,0.637085,0.670894,0.673424
8,0.380500,1.379842,0.639706,0.639330,0.662087,0.672775


                           model  learning_rate  batch_size  accuracy  \
0   SpanBERT/spanbert-base-cased       0.000005          16  0.610080   
1   SpanBERT/spanbert-base-cased       0.000005          32  0.618037   
2   SpanBERT/spanbert-base-cased       0.000010          16  0.694960   
3   SpanBERT/spanbert-base-cased       0.000010          32  0.628647   
4   SpanBERT/spanbert-base-cased       0.000020          16  0.639257   
5   SpanBERT/spanbert-base-cased       0.000020          32  0.668435   
6   SpanBERT/spanbert-base-cased       0.000030          16  0.618037   
7   SpanBERT/spanbert-base-cased       0.000030          32  0.652520   
8   SpanBERT/spanbert-base-cased       0.000050          16  0.612732   
9   SpanBERT/spanbert-base-cased       0.000050          32  0.625995   
10                  roberta-base       0.000005          16  0.503979   
11                  roberta-base       0.000005          32  0.503979   
12                  roberta-base       0.000010    


---

These final results are further discussed in the Conference Paper Draft. An interesting thing to note here is that there is a sudden drop of overall performance comparing the logs displayed from the validation set and the final test set - which probably might suggest overfitting due to the small number of training and dev samples